# Análisis Exploratorio y Regresión Lineal
Stefano Aragoni & Carol Arévalo

## Importar las librerías

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import datetime

from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

## Parte 1
### 1.1. Haga una exploración rápida de sus datos para eso haga un resumen de su dataset.

El conjunto de datos proporcionado contiene información sobre juegos de béisbol. Las columnas incluyen:

In [12]:
data = pd.read_csv("baseball_reference_2016_scrape.csv")
pd.set_option('display.max_columns', None)

# Obtener los nombres de las columnas
data.columns.values.tolist()

['attendance',
 'away_team',
 'away_team_errors',
 'away_team_hits',
 'away_team_runs',
 'boxscore_url',
 'date',
 'field_type',
 'game_duration',
 'game_type',
 'home_team',
 'home_team_errors',
 'home_team_hits',
 'home_team_runs',
 'other_info_string',
 'start_time',
 'venue']

Asimismo, a continuación se presenta la cantidad de columas y filas del dataset.

In [19]:
#Obtener el número de filas y columnas
print(data.shape)

(2463, 17)


A continuación se presenta el resumen de las columnas mencionadas. Esta imagen presenta información sobre la cantidad de entradas (no vacías) relacionadas a cada columna, así como el tipo de información que almacena.

In [14]:
# Obtener información general sobre el conjunto de data
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463 entries, 0 to 2462
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   attendance         2463 non-null   object 
 1   away_team          2463 non-null   object 
 2   away_team_errors   2463 non-null   int64  
 3   away_team_hits     2463 non-null   int64  
 4   away_team_runs     2463 non-null   int64  
 5   boxscore_url       2463 non-null   object 
 6   date               2463 non-null   object 
 7   field_type         0 non-null      float64
 8   game_duration      2463 non-null   object 
 9   game_type          2460 non-null   object 
 10  home_team          2463 non-null   object 
 11  home_team_errors   2463 non-null   int64  
 12  home_team_hits     2463 non-null   int64  
 13  home_team_runs     2463 non-null   int64  
 14  other_info_string  2463 non-null   object 
 15  start_time         2463 non-null   object 
 16  venue              2463 

Asimismo, se presenta un resumen estadístico de las columnas numéricas, que incluye promedio, desviación estándar, y otros datos.

In [15]:
# Obtener un resumen estadístico de las columnas numéricas
print(data.describe())

       away_team_errors  away_team_hits  away_team_runs  field_type  \
count       2463.000000     2463.000000     2463.000000         0.0   
mean           0.580593        8.764515        4.413723         NaN   
std            0.793391        3.511581        3.104556         NaN   
min            0.000000        1.000000        0.000000         NaN   
25%            0.000000        6.000000        2.000000         NaN   
50%            0.000000        8.000000        4.000000         NaN   
75%            1.000000       11.000000        6.000000         NaN   
max            5.000000       22.000000       21.000000         NaN   

       home_team_errors  home_team_hits  home_team_runs  
count       2463.000000     2463.000000     2463.000000  
mean           0.585871        8.611855        4.519691  
std            0.805542        3.436965        3.111572  
min            0.000000        0.000000        0.000000  
25%            0.000000        6.000000        2.000000  
50%         

Finalmente, a continuación se presentan los nombres de las respectivas columnas, así como los primeros 5 datos almacenados en el dataset.

In [16]:
head = data.head()
print(head)

  attendance              away_team  away_team_errors  away_team_hits  \
0   40,030']          New York Mets                 1               7   
1   21,621']  Philadelphia Phillies                 0               5   
2   12,622']        Minnesota Twins                 0               5   
3   18,531']   Washington Nationals                 0               8   
4   18,572']       Colorado Rockies                 1               8   

   away_team_runs                                       boxscore_url  \
0               3  https://www.baseball-reference.com/boxes/KCA/K...   
1               2  https://www.baseball-reference.com/boxes/CIN/C...   
2               2  https://www.baseball-reference.com/boxes/BAL/B...   
3               3  https://www.baseball-reference.com/boxes/ATL/A...   
4               4  https://www.baseball-reference.com/boxes/ARI/A...   

                       date  field_type game_duration             game_type  \
0     Sunday, April 3, 2016         NaN        : 

### 1.2 Diga el tipo de cada una de las variables del dataset (cualitativa o categórica, cuantitativa continua, cuantitativa discreta)

![TIPOS VARIABLE](image1.png "Tipos Variable")


### 1.3. Incluya los gráficos exploratorios siendo consecuentes con el tipo de variable que están representando.